In [1]:
#Updated Model from the initial one created 

In [2]:
#Updates include the following:
# 1. clean data
# 2. connection to a database that houses the csv file
# 3. Scaled the data
# 4. Define the model- deep neural net
# 5. Added 3 hidden layers 
# 6. Compile the model 
# 7. Train the model
# 8. Evaluate the loss and accuracy metrics


In [3]:
# Future changes:
# 1. Adding more hidden layers 
# 2. Running the model with a different number of features to see if accuracy and loss are affected
# 3. Altering aspects such as activation functions to see if model accuracy improves. 

In [4]:
# Import our dependencies
import pandas as pd
import matplotlib as plt
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine

In [7]:
# Postgres username, password, and database name (change accordingly)
postgres_address = 'localhost'
postgres_port = '5432' 
postgres_username = 'postgres' 
postgres_password = '********'
postgres_dbname = 'cardio_disease_db'
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=postgres_username,
                        password=postgres_password,
                        ipaddress=postgres_address,
                        port=postgres_port,
                        dbname=postgres_dbname))
# Create the connection
cnx = create_engine(postgres_str)

In [11]:
# Load the data
cardio_train_df = pd.read_sql_query('''SELECT * FROM cardio_data_clean;''', cnx)
cardio_train_df.head()

,id_no,age,gender,height,weight,ap_hi,ap_lo,cholesterol,glucose,smoke,alcohol,active,cardio
0,0,50,2,168,62.0,110,80,1,1,False,False,True,False
1,1,55,1,156,85.0,140,90,3,1,False,False,True,True
2,2,52,1,165,64.0,130,70,3,1,False,False,False,True
3,3,48,2,169,82.0,150,100,1,1,False,False,True,True
4,4,48,1,156,56.0,100,60,1,1,False,False,False,False


In [6]:
# Define feature data
# features: age, gender, height, weight, ap_hi, 
# ap_lo, cholesterol, gluc, smoke, alco, active 
X = cardio_train_df.drop(['id','cardio'],1).values

In [7]:
# Define target data
# cardio (if the person has cardiovascular disease or not) 
y = cardio_train_df['cardio']

In [8]:
# Create training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [9]:
# Create a StandardScaler instance
X_scaler = StandardScaler()

In [10]:
# Fit the StandardScaler
X_scaler.fit(X_train)

StandardScaler()

In [11]:
# Scale the features data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  15
hidden_nodes_layer2 = 8
hidden_nodes_layer3 = 8

nn_model = tf.keras.models.Sequential()

In [13]:
# First hidden layer
nn_model.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

In [14]:
# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

In [15]:
# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

In [16]:
# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [17]:
# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 15)                180       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 128       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 389
Trainable params: 389
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Train the model
fit_model = nn_model.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
1582/1582 [==============================] - 1s 643us/step - loss: 0.5953 - accuracy: 0.6973
Epoch 2/100
1582/1582 [==============================] - 1s 652us/step - loss: 0.5551 - accuracy: 0.7229
Epoch 3/100
1582/1582 [==============================] - 1s 665us/step - loss: 0.5473 - accuracy: 0.7307
Epoch 4/100
1582/1582 [==============================] - 1s 647us/step - loss: 0.5439 - accuracy: 0.7313
Epoch 5/100
1582/1582 [==============================] - 1s 669us/step - loss: 0.5518 - accuracy: 0.7235
Epoch 6/100
1582/1582 [==============================] - 1s 681us/step - loss: 0.5434 - accuracy: 0.7313
Epoch 7/100
1582/1582 [==============================] - 1s 672us/step - loss: 0.5462 - accuracy: 0.7271
Epoch 8/100
1582/1582 [==============================] - 1s 653us/step - loss: 0.5443 - accuracy: 0.7344
Epoch 9/100
1582/1582 [==============================] - 1s 616us/step - loss: 0.5449 - accuracy: 0.7333
Epoch 10/100
1582/1582 [==============================]